In [18]:

import numpy as np
import matplotlib.pyplot as plt

import torch

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import RidgeClassifier
from sklearn.kernel_approximation import RBFSampler
from sklearn.preprocessing import LabelBinarizer

import time
from gpu_energy.power_info import EnergyMonitor


import synthetic_opu
from multiple_regression_solver import MultipleRegressionSolver



In [19]:
# (train_data, train_labels), (test_data, test_labels) = FashionMNIST()
#dataset_folder = "../../datasets/export" 
dataset_folder = "../Datasets"
train_data = np.load(dataset_folder+'/fashion_mnist/numpy/train_data_fashion_mnist.npy').astype('uint8')
test_data = np.load(dataset_folder+'/fashion_mnist/numpy/test_data_fashion_mnist.npy').astype('uint8')
train_labels = np.load(dataset_folder+'/fashion_mnist/numpy/train_targets_fashion_mnist.npy').astype('uint8')
test_labels = np.load(dataset_folder+'/fashion_mnist/numpy/test_targets_fashion_mnist.npy').astype('uint8')

# Convert one-hot to integers
train_labels = np.argmax(train_labels, axis=1)
test_labels = np.argmax(test_labels, axis=1)

D = train_data[0].reshape(-1).shape[0]

# Flatten the images
train_data = train_data.reshape(-1, D)
test_data = test_data.reshape(-1, D)

In [21]:
EM = EnergyMonitor()

Detected GPU(s): GPU 0 (161.69 W), GPU 1 (30.26 W), GPU 2 (32.2 W).
Stop monitoring GPUs.


## Project the data

In [22]:
def threshold_binarize(data, threshold):
    data_bin = np.where(data>threshold, 1, 0).astype('uint8')
    return data_bin

In [23]:
# fashion mnist has values between 0 and 255
threshold = 10

train_data_bin = threshold_binarize(train_data, threshold)
test_data_bin = threshold_binarize(test_data, threshold)

In [24]:
train_data_bin = threshold_binarize(train_data, 10)
test_data_bin = threshold_binarize(test_data, 10)
# we need to work with flot32 or float64 for some reason.
# otherwise numpy freezes
all_data = np.vstack([train_data_bin, test_data_bin]).astype('float32')

In [25]:
out_dim = 10000
ener1 = EM.energy()
data_proj = synthetic_opu.project_big_np_matrix(all_data, out_dim=10000, chunk_size=5000, projection='opu',
                          framework='pytorch', dtype=torch.FloatTensor, cuda=True)

Starting GPU energy monitoring in background... OK.
Processing chunk of size: torch.Size([5000, 784])
Processing chunk of size: torch.Size([5000, 784])
Processing chunk of size: torch.Size([5000, 784])
Processing chunk of size: torch.Size([5000, 784])
Processing chunk of size: torch.Size([5000, 784])
Processing chunk of size: torch.Size([5000, 784])
Processing chunk of size: torch.Size([5000, 784])
Processing chunk of size: torch.Size([5000, 784])
Processing chunk of size: torch.Size([5000, 784])
Processing chunk of size: torch.Size([5000, 784])
Processing chunk of size: torch.Size([5000, 784])
Processing chunk of size: torch.Size([5000, 784])
Processing chunk of size: torch.Size([5000, 784])
Processing chunk of size: torch.Size([5000, 784])
Total time elapsed (seconds): 14.751839876174927
Time per chunk (seconds): 1.0537028482982091


In [26]:
train_proj = data_proj[:len(train_data_bin)]
test_proj = data_proj[len(train_data_bin):]

## Regression on the features

In [27]:
# like one-hot encoding with 0 corresponding to -1
label_binarizer = LabelBinarizer(pos_label=1, neg_label=-1)
train_labels_bin = label_binarizer.fit_transform(train_labels)
test_labels_bin = label_binarizer.fit_transform(test_labels)

In [28]:
solver = MultipleRegressionSolver(train_proj, train_labels_bin, batch_size=128, cuda=True)

In [29]:
ener2 = EM.energy()
print(ener2-ener1)

 GPU | Energy (J) | Avg. pow (W)
   0 |    1265.30 |       126.34
   2 |     325.62 |        32.51
   1 |     304.28 |        30.38
 over a period of 10015 milliseconds.


## Synthetic OPU

### 1K Features

In [14]:
optimizer = torch.optim.Adam(solver.model.parameters(), lr=1e-3)

In [15]:
coefficients = solver.fit(optimizer, epochs=50)

Epoch: 0 Loss: 58.79425048828125 Accuracy: 12.5
Epoch: 1 Loss: 1.1396335363388062 Accuracy: 49.21875
Epoch: 2 Loss: 0.5425878167152405 Accuracy: 78.125
Epoch: 3 Loss: 0.757494330406189 Accuracy: 69.53125
Epoch: 4 Loss: 4.021361351013184 Accuracy: 20.3125
Epoch: 5 Loss: 2.530369281768799 Accuracy: 41.40625
Epoch: 6 Loss: 2.572263717651367 Accuracy: 38.28125
Epoch: 7 Loss: 1.3776670694351196 Accuracy: 52.34375
Epoch: 8 Loss: 1.5671007633209229 Accuracy: 49.21875
Epoch: 9 Loss: 5.898340702056885 Accuracy: 15.625
Epoch: 10 Loss: 4.693675518035889 Accuracy: 35.9375
Epoch: 11 Loss: 1.8051496744155884 Accuracy: 51.5625
Epoch: 12 Loss: 1.6415901184082031 Accuracy: 35.9375
Epoch: 13 Loss: 1.3518457412719727 Accuracy: 45.3125
Epoch: 14 Loss: 5.09631872177124 Accuracy: 14.84375
Epoch: 15 Loss: 1.6905406713485718 Accuracy: 43.75
Epoch: 16 Loss: 5.776478290557861 Accuracy: 15.625
Epoch: 17 Loss: 1.8163782358169556 Accuracy: 56.25
Epoch: 18 Loss: 13.81782054901123 Accuracy: 32.8125
Epoch: 19 Loss: 3

In [16]:
solver.classification_score(test_proj, test_labels_bin)

Accuracy: 59.123333333333335


59.123333333333335

In [17]:
ener3 = EM.energy()
print(ener3-ener2)

 GPU | Energy (J) | Avg. pow (W)
   0 |   35128.76 |       124.24
   2 |    9199.81 |        32.54
   1 |    8529.90 |        30.17
 over a period of 282753 milliseconds.


### 10K Features

In [17]:
optimizer = torch.optim.Adam(solver.model.parameters(), lr=1e-4)

In [18]:
coefficients = solver.fit(optimizer, epochs=50)

Epoch: 0 Loss: 19.49797248840332 Accuracy: 12.5
Epoch: 1 Loss: 0.9232721328735352 Accuracy: 53.125
Epoch: 2 Loss: 0.6023041605949402 Accuracy: 64.84375
Epoch: 3 Loss: 0.4681790769100189 Accuracy: 72.65625
Epoch: 4 Loss: 0.4207070469856262 Accuracy: 70.3125
Epoch: 5 Loss: 0.35498887300491333 Accuracy: 75.0
Epoch: 6 Loss: 0.35204654932022095 Accuracy: 85.9375
Epoch: 7 Loss: 0.3081686198711395 Accuracy: 83.59375
Epoch: 8 Loss: 0.33145350217819214 Accuracy: 78.90625
Epoch: 9 Loss: 0.36751434206962585 Accuracy: 76.5625
Epoch: 10 Loss: 0.23600712418556213 Accuracy: 86.71875
Epoch: 11 Loss: 0.2516993284225464 Accuracy: 83.59375
Epoch: 12 Loss: 0.2738135755062103 Accuracy: 80.46875
Epoch: 13 Loss: 0.21624425053596497 Accuracy: 88.28125
Epoch: 14 Loss: 0.22382351756095886 Accuracy: 84.375
Epoch: 15 Loss: 0.29099106788635254 Accuracy: 80.46875
Epoch: 16 Loss: 0.22866110503673553 Accuracy: 86.71875
Epoch: 17 Loss: 0.24393053352832794 Accuracy: 79.6875
Epoch: 18 Loss: 0.22331933677196503 Accuracy:

In [19]:
solver.classification_score(test_proj, test_labels_bin)

Accuracy: 88.74666666666667


88.74666666666667

### 100K Features

In [30]:
optimizer = torch.optim.Adam(solver.model.parameters(), lr=1e-5)

In [31]:
coefficients = solver.fit(optimizer, epochs=50)

Epoch: 0 Loss: 35.02031326293945 Accuracy: 21.09375
Epoch: 1 Loss: 1.1676433086395264 Accuracy: 54.6875
Epoch: 2 Loss: 0.6797206401824951 Accuracy: 66.40625
Epoch: 3 Loss: 0.5919178128242493 Accuracy: 66.40625
Epoch: 4 Loss: 0.5301603078842163 Accuracy: 67.96875
Epoch: 5 Loss: 0.4421910345554352 Accuracy: 79.6875
Epoch: 6 Loss: 0.4331490397453308 Accuracy: 68.75
Epoch: 7 Loss: 0.34248507022857666 Accuracy: 78.125
Epoch: 8 Loss: 0.33059290051460266 Accuracy: 74.21875
Epoch: 9 Loss: 0.37924784421920776 Accuracy: 74.21875
Epoch: 10 Loss: 0.2868381440639496 Accuracy: 81.25
Epoch: 11 Loss: 0.2873137593269348 Accuracy: 80.46875
Epoch: 12 Loss: 0.2708255350589752 Accuracy: 80.46875
Epoch: 13 Loss: 0.3619306683540344 Accuracy: 74.21875
Epoch: 14 Loss: 0.2695775032043457 Accuracy: 82.8125
Epoch: 15 Loss: 0.36703965067863464 Accuracy: 64.0625
Epoch: 16 Loss: 0.25032204389572144 Accuracy: 84.375
Epoch: 17 Loss: 0.22382381558418274 Accuracy: 84.375
Epoch: 18 Loss: 0.25698143243789673 Accuracy: 82.

In [32]:
solver.classification_score(test_proj, test_labels_bin)

Accuracy: 91.075


91.075